# Miljødataanalyse - Del 1, Oppgave 2.

In [1]:
# Sjekk directory
import os
print(os.getcwd())

c:\Users\betja\OneDrive - NTNU\2025VårProgrammering\Milj-dataanalyse43\notebooks


In [2]:
import requests
import os
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv('API_KEY')

city = 'Oslo'

api_key = os.getenv('API_KEY')

url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric'

res = requests.get(url)
data = res.json()

humidity = data['main']['humidity']
pressure = data['main']['pressure']
wind = data['wind']['speed']
description = data['weather'][0]['description']
temp = data['main']['temp']

print('Temperature:',temp,'°C')
print('Wind:',wind)
print('Pressure: ',pressure)
print('Humidity: ',humidity)
print('Description:',description)


Temperature: 8.31 °C
Wind: 2.06
Pressure:  1017
Humidity:  50
Description: broken clouds
